# Data cleaning and merging

In [ ]:
import pandas as pd

#All the data has city/province names followed by soum/district
df1 = pd.read_csv("/content/Number_of_companies.csv")  # e.g., soum-level population
df2 = pd.read_excel("/content/Population_age_districts.xlsx", skiprows = 3)
df3 = pd.read_excel("/content/Mongolian banks' loans 2019-2024.xlsx", skiprows = 5, sheet_name = "2024")
df4 = pd.read_excel("/content/Mongolian bank's current accounts and balances 2019-2024.xlsx", skiprows = 5, sheet_name = "2024")
df5 = pd.read_excel("/content/Mongolian banks' deposits 2015-2024.xlsx", skiprows = 5, sheet_name = "2024")

df3 = df3.rename(columns={
    'Үүнээс:': 'Normal',
    'Unnamed: 5': 'Delinquent',
    'Unnamed: 6': 'Overdue'
})

df5 = df5.rename(columns={
    'From which:': 'Timeless_deposits',
    'Unnamed: 5': 'Time_deposits',
    #'Unnamed: 6': 'Overdue'
})


print("df1 columns:", df1.columns.tolist())
print("df2 columns:", df2.columns.tolist())
print("df3 columns:", df3.columns.tolist())
print("df4 columns:", df4.columns.tolist())
print("df5 columns:", df5.columns.tolist())


cols2 = ['City', 'District', 'Grand Total', 'Total_kids', 'Total_working', 'Total_olders', 'Percentage_kids', 'Percentage_working', 'Percentage_olders']
cols3 = ['City', 'District', 'Total_loan_balance','Total_loans_accounts', 'Normal', 'Delinquent', 'Overdue']
cols5 = ['City', 'District', 'Total_deposits', 'Total_deposits_accounts', 'Time_deposits', 'Timeless_deposits']

df1 = df1.dropna(subset=['City'])
df2 = df2.dropna(subset=['City'])
df2 = df2[cols2]
df3 = df3[cols3]
df5 = df5[cols5]

for df in [df1, df2, df3, df4, df5]:
    df['City'] = df['City'].astype(str).str.strip().str.lower()
    df['District'] = df['District'].astype(str).str.strip().str.lower()

merged_df = pd.merge(df1, df2, on=['City', 'District'], how='inner')
merged_df = pd.merge(merged_df, df3, on=['City', 'District'], how='inner')
merged_df = pd.merge(merged_df, df4, on=['City', 'District'], how='inner')
merged_df = pd.merge(merged_df, df5, on=['City', 'District'], how='inner')


print("Original df1:", len(df1))
print("Original df2:", len(df2))
print("Merged df:", len(df))

merged_df


df1 columns: ['City', 'District', 'Companies']
df2 columns: ['City', 'District', '0-4', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70+', '5-9', '10-14', 'Grand Total', 'Total_kids', 'Total_working', 'Total_olders', 'Percentage_kids', 'Percentage_working', 'Percentage_olders']
df3 columns: ['Number', 'City', 'District', 'Total_loan_balance', 'Normal', 'Delinquent', 'Overdue', 'Total_loans_accounts', 'Үүнээс:.1', 'Unnamed: 9', 'Unnamed: 10']
df4 columns: ['Number', 'City', 'District', 'Total_balances', 'Total_current_accounts']
df5 columns: ['Number', 'City', 'District', 'Total_deposits', 'Timeless_deposits', 'Time_deposits', 'Total_deposits_accounts', 'From which:.1', 'Unnamed: 8']
Original df1: 339
Original df2: 342
Merged df: 374


<ipython-input-11-1068154519>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['City'] = df['City'].astype(str).str.strip().str.lower()
<ipython-input-11-1068154519>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['District'] = df['District'].astype(str).str.strip().str.lower()


,City,District,Companies,Grand Total,Total_kids,Total_working,Total_olders,Percentage_kids,Percentage_working,Percentage_olders,...,Normal,Delinquent,Overdue,Number,Total_balances,Total_current_accounts,Total_deposits,Total_deposits_accounts,Time_deposits,Timeless_deposits
0,говь-алтай,алтай,21,1909,568.0,1253.0,88.0,0.297538,0.656365,0.046097,...,39715.606129,158.248099,207.518385,1.1.2.2,5375.042866,11871,15676.125117,5213.0,12871.332484,2804.792633
1,говь-алтай,баян-уул,34,3026,850.0,2027.0,149.0,0.280899,0.669861,0.049240,...,10824.832192,41.684376,8.784859,1.1.2.3,2084.376542,4858,6991.142922,2142.0,5789.870931,1201.271991
2,говь-алтай,бигэр,44,2204,616.0,1464.0,124.0,0.279492,0.664247,0.056261,...,8060.529232,21.461958,6.477661,1.1.2.4,2158.305947,3701,5115.532846,1490.0,4351.552852,763.979993
3,говь-алтай,бугат,22,2139,621.0,1410.0,108.0,0.290323,0.659187,0.050491,...,5862.357803,42.508236,30.616131,1.1.2.5,1630.978377,3916,7613.467698,1532.0,6342.170473,1271.297225
4,говь-алтай,дэлгэр,49,3032,802.0,2069.0,161.0,0.264512,0.682388,0.053100,...,14337.50642,126.604846,124.117105,1.1.2.7,1950.544576,5504,5865.064568,1949.0,4751.227161,1113.837407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,хэнтий,батноров,73,5731,1687.0,3678.0,366.0,0.294364,0.641773,0.063863,...,15334.657081,21.593898,36.717167,1.4.3.2,1874.144549,5170,5572.657733,1653.0,4432.277263,1140.38047
304,хэнтий,баян-овоо,26,1934,601.0,1228.0,105.0,0.310755,0.634953,0.054292,...,8059.718021,32.783742,40.236375,1.4.3.6,1060.939288,3103,2603.308827,1317.0,2023.645451,579.663376
305,хэнтий,мөрөн,32,1845,459.0,1264.0,122.0,0.248780,0.685095,0.066125,...,10121.684168,58.793751,32.894776,1.4.3.15,899.805254,2717,1179.384067,728.0,847.258737,332.12533
306,хэнтий,хэрлэн,570,24764,8202.0,15051.0,1511.0,0.331207,0.607777,0.061016,...,113961.014707,1023.566577,959.803574,1.4.3.1,16899.453091,34770,55404.753895,16685.0,43767.090607,11637.663288
